 # Opis notatnika
 Zmierzamy do końca analizy danych, które zostały nam udostępnione. Ten krok dodaje jeszcze więcej informacji do naszego wyjściowego zbioru. Tym razem sprawdzimy między innymi to, czy opóźnienia lotów zależne są od trasy czy warunków pogodowych.

 Zanim jednak do tego przejdziemy, należy, podobnie jak w poprzednich krokach, skonfigurować odpowiednio notatnik.
 
 W tej części warsztatu ponownie wcielasz się w rolę Analiyka Danych, którego zadaniem jest wykonanie analizy eksplotacyjnej zbioru danych - jedno z wymagań dostarczonych przez klienta.

 Tutaj zaimportuj wymagane biblioteki

In [ ]:
import mysql.connector as sql
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

 ## Połączenie z bazą danych
 Tutaj uzupełnij konfigurację połączenia

In [ ]:
username = 'postgres'
password = 'postgres'

host = 'localhost'
database = 'airlines'
port = 5432

 Tutaj stwórz zmienną engine, która zostanie użyta do połączenia z bazą danych

In [ ]:
url = f'postgresql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

 Tutaj uzupełnij implementację metody `read_sql_table`

In [ ]:
def read_sql_table(table_name):
    try:
        # Wykonaj zapytanie SQL, aby pobrać całą zawartość tabeli.
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, con=engine)
        print(f"Data from table '{table_name}' loaded successfully.")
        return df
    except Exception as e:
        print(f"Failed to load data from table '{table_name}': {e}")
        return None

 Tutaj zaczytaj zapisaną wcześniej ramkę danych `flight_df` do zmniennej o takiej samej nazwie

In [ ]:
flight_df = pd.read_csv('flight_df_02.csv')

 # Wzbogacenie o `airport_list`
 Wczytaj do obszaru roboczego tabelę `airport_list` używając procedury `read_sql_table`. Wykonaj poniższe ćwiczenia:  
 1. Sprawdź, czy klucz `origin_airport_id` jest unikalny, tj. nie ma dwóch takich samych wartości w kolumnie `origin_airport_id`.  
 1. Jeżeli duplikaty występują, usuń je w najdogodniejszy dla Ciebie sposób.  
 1. Jeśli duplikaty nie występują, złącz ramki `airport_list_df` wraz z aktualną `flight_df`, używając kolumny `origin_airport_id` oraz złączenia typu `LEFT JOIN`. Z ramki `airport_list_df` interesuje nas dodanie kolumny `origin_city_name`.  
 1. Dodatkowo dokonaj jeszcze raz złączenia ramki `flight_df` z `airport_list_df`, tym razem jednak złącz kolumnę `destination_airport_id` wraz z `origin_airport_id`. Podobnie jak wcześniej, interesuje nas kolumna `origin_city_name`, jedank ona powinna zostać wyświetlona jako `destination_city_name`

 Tutaj wczytaj ramkę `airport_list_df`

In [ ]:
airport_list_df = pd.read_sql_table('airport_list', con=engine)

 Tutaj sprawdż, czy występują duplikaty dla kolumny `origin_airport_id`

In [ ]:
airport_list_df['origin_airport_id'].duplicated().sum()

 Tutaj usuń duplikaty – jeśli występują

In [ ]:
airport_list_df.info()

In [ ]:
flight_df.info()

 Tutaj dokonaj złączenia ramki `flight_df` oraz `airport_list_df` używając `origin_airport_id`

In [ ]:
flight_df = pd.merge(
    flight_df,
    airport_list_df,
    how='left',
    left_on='origin_airport_id',
    right_on='origin_airport_id'
)

In [ ]:
flight_df.columns

 Tutaj dokonaj złączenia ramki `flight_df` oraz `airport_list_df` używając `destination_airport_id`

In [ ]:
flight_df = pd.merge(
    flight_df,
    airport_list_df[['origin_airport_id', 'origin_city_name']],
    how='left',
    left_on='dest_airport_id',
    right_on='origin_airport_id'
)

In [ ]:
flight_df.columns

In [ ]:
flight_df = flight_df.rename(columns={'origin_city_name_y': 'destination_city_name'})
flight_df = flight_df.rename(columns={'origin_city_name_x': 'origin_city_name'})
flight_df = flight_df.rename(columns={'origin_airport_id_y': 'destination_airport_id'})
flight_df = flight_df.rename(columns={'origin_airport_id_x': 'origin_airport_id'})

In [ ]:
flight_df.info()

### Sprawdzenie
Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
assert 'origin_city_name' in flight_df.columns, 'Brak kolumny `origin_city_name` w ramce flight_df'
assert 'destination_city_name' in flight_df.columns, 'Brak kolumny `destination_city_name` w ramce flight_df'

flight_df_expected_rows_amount = 1057391
assert flight_df.shape[0] == flight_df_expected_rows_amount, 'Ups, zmieniła się liczba wierszy...'

 ## Analiza według lotnisk oraz tras
 Wykonaj poniższe polecenia:  
 1. Wyznacz lotniska, z których **odlatywało** najwięcej samolotów. Wynik zapisz do ramki `top_airports_origin_df`.
 1. Wyznacz lotnika, na których najwięcej lotów **się kończyło**. Wynik zapisz do ramki `top_airports_destination_df`.  
 1. Wyznacz najczęściej uczęszczaną trasę, wynik zapisz do ramki `top_route_df`.  
 1. Przy założeniu, że reprezentatywna liczba lotów na trasie wynosi ponad 500, wyznacz dodatkowo top 10:  
     - tras z **najmniejszym odsetkiem opóźnień**, wynik zapisz do ramki `least_route_delays_df`.  
     - tras z **największym odsetkiem opóźnień**, wynik zapisz do ramki `top_route_delays_df`.

 Tutaj wyznacz ramkę `top_airports_origin_df`

In [ ]:
top_airports_origin_df = flight_df.groupby('origin_city_name')['id_x'].count().reset_index(name='number_of_flights')
top_airports_origin_df = top_airports_origin_df.sort_values(by='number_of_flights', ascending=False)
top_airports_origin_df

 Tutaj wyznacz ramkę `top_airports_destination_df`

In [ ]:
top_airports_destination_df = flight_df.groupby('destination_city_name')['id_x'].count().reset_index(name='number_of_flights')
top_airports_destination_df = top_airports_destination_df.sort_values(by='number_of_flights', ascending=False)
top_airports_destination_df

 # Wzbogacenie o dane pogodowe
 Używając procedury `read_sql_table`, wczytaj tabelę `airport_weather` do ramki `airport_weather_df`. Następnie wykonaj następujące polecenia:  
 1. Pozostaw w ramce tylko następujące kolumny: `['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']`.  
 1. Połącz ramki `airport_list_df` wraz z `airport_weather_df` po odpowiedniej kolumnie używając takiego złączenia, aby w wyniku usunąć te wiersze (lotniska), które nie posiadają danych pogodowych. Dodatkowo, upewnij się, że zostanie tylko dodana kolumna `origin_airport_id`.

 Tutaj wczytaj ramkę `airport_weather`

In [ ]:
airport_weather_df = pd.read_sql_table('airport_weather', con=engine)

 Tutaj oczyść ramkę `airport_weather_df` z nadmiarowych kolumn

In [ ]:
selected_columns = ['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']
airport_weather_df = airport_weather_df[selected_columns]
airport_weather_df.head()

 Tutaj połącz ramki `airport_list_df` oraz `airport_weather_df` aktualizując `airport_weather_df`

In [ ]:
airport_list_df.head()

In [ ]:
airport_weather_df = pd.merge(
    airport_list_df[['origin_airport_id', 'name']],  # Kolumna 'name' oraz 'origin_airport_id' z airport_list_df
    airport_weather_df,                            # Cała ramka airport_weather_df
    how='inner',                                   # Inner join, aby usunąć lotniska bez danych pogodowych
    left_on='name',                               # Kolumna 'name' w airport_list_df
    right_on='name'                               # Kolumna 'name' w airport_weather_df
)

In [ ]:
airport_weather_df.head()

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
airport_weather_df_expected_shape = (43394, 9)
airport_weather_df_shape = airport_weather_df.shape

assert airport_weather_df_expected_shape == airport_weather_df_shape, \
  f'Nieodpowiedni wymiar ramki airport_weather_df, oczekiwano (wierszy, kolumn): {airport_weather_df_expected_shape}'


 ## Połączenie `airport_weather_df` oraz `flight_df`
 W celu złączenia ramek `airport_weather_df` oraz `flight_df` wykonaj następujące kroki:  
 1. w ramce `aiport_weather_df` występuje kolumna `date`, zrzutuj ją na typ `DATETIME`.  
 1. w ramce `flight_df` należy stworzyć nową kolumnę o nazwie `date`. W tym celu:  
 	- złącz kolumny `month`, `day_of_month` oraz `year` razem, użyj następującego formatu daty: `YYYY-MM-DD`.
 	- zrzutuj kolumnę `date` na typ `DATETIME`.  
 1. złącz ramki używając odpowiedniego klucza, wynik złączenia zapisz do ramki `flight_df`. Użyj złącznia typu `LEFT JOIN`.

 > Dlaczego istotne jest zachowanie typów przy złączeniu?

W trakcie pracy możesz posłużyć się następującymi artykułami z `LMS`:
 - `Python - analiza danych > Dzień 6 - Pandas > Merge`
 - `Python - analiza danych > Dzień 6 - Pandas > Praca z datetime`
 - Dokumentacje metody `to_datetime`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)
 - Dostępne formaty dat: [klik](https://www.programiz.com/python-programming/datetime/strftime) - sekcja `Format Code List`

 Tutaj zrzutuj kolumnę `date` na `DATETIME` w ramce `airport_weather_df`

In [ ]:
airport_weather_df['date'] = pd.to_datetime(airport_weather_df['date'])

 Tutaj stwórz kolumnę `date` w ramce `flight_df`. Pamiętaj, aby była ona również typu `DATETIME`.

In [ ]:
flight_df['date'] = pd.to_datetime(
    flight_df[['year', 'month', 'day_of_month']].astype(int).astype(str).agg('-'.join, axis=1),
    format='%Y-%m-%d'
)

 Tutaj złącz tabeli `airport_weather_df` oraz `flight_df`

In [ ]:
flight_df = pd.merge(
    flight_df,
    airport_weather_df[['origin_airport_id', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']],
    how='left',
    on=['origin_airport_id', 'date']
)

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
flight_df_expected_rows_amount = 1057391
assert flight_df.shape[0] == flight_df_expected_rows_amount, 'Ups, zmieniła się liczba wierszy...'

# Praca samodzielna
Używając `flight_df` zbadaj hipotezę o tym, że temperatura maksymalna wpływa na **odsetek** opóźnień lotów (kolumna `tmax`).  

Przy wykonywaniu tego zadania masz pełną dowolność, jednak powinno składać się conajmniej z następujących elementów:
- sprawdzenie, czy zmienna posiada obserwacje odstające,
- oczyszczenie danych o ile konieczne,
- przedstawienie w formie tabeli czy wzrost danej zmiennej powoduje zmianę w odsetku opóźnień lotów,
- wizualizację stworzonej wcześniej tabeli w formie wykresu,
- krótkiego opisu wyników w komórce markdown.

 ## Analiza dla kolumny `tmax`

**BEZ OCZYSZCZANIA DANYCH Z "WARTOŚCI SKRAJNYCH"**

In [ ]:
flight_df['tmax'].describe().round(2)

In [ ]:
flight_df['tmax'].median()

Średnia i mediana są zbliżone - przesłanka do tego, że nie występują wartości odstające.

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(flight_df['tmax'].dropna(), bins=50, kde=True)
plt.title('Rozkład temperatury maksymalnej (tmax)')
plt.xlabel('Temperatura maksymalna (°F)')
plt.ylabel('Częstotliwość')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x=flight_df['tmax'])
plt.title('Wykres pudełkowy temperatury maksymalnej (tmax)')
plt.xlabel('Temperatura maksymalna (°F)')
plt.show()

Powyższe wykresy potwierdzają, że nie ma potrzeby oczyszczania zmiennej z wartości odstających, natomiast dla pewności i tak ją przeprowadzimy w kolejnym kroku.

In [ ]:
flight_delays_by_tmax_df = flight_df.groupby('tmax')['is_delayed'].mean().reset_index()

In [ ]:
flight_delays_by_tmax_df['delayed_percentage'] = flight_delays_by_tmax_df['is_delayed'] * 100

In [ ]:
flight_delays_by_tmax_df = flight_delays_by_tmax_df.drop(columns=['is_delayed'])

In [ ]:
flight_delays_by_tmax_df

In [ ]:
plt.figure(figsize=(14, 7))
sns.scatterplot(
    data=flight_delays_by_tmax_df,
    x='tmax',
    y='delayed_percentage',
    alpha=0.5,  # Ustawia przezroczystość punktów
    edgecolor=None  # Opcjonalne: usunięcie obramowania punktów
)

plt.title('Temperature vs Percentage of Delayed Flights')
plt.xlabel('Temperature (°F)')
plt.ylabel('Percentage of Delayed Flights (%)')
plt.grid(True)

Wzrost zmiennej tmax nie wpływa na zwiększenie odsetka opóźnionych lotów. Można zauważyć ze ta zależność jest odwrotna.

**Z OCZYSZCZANIEM DANYCH Z "WARTOŚCI SKRAJNYCH"**

In [ ]:
Q1 = flight_df['tmax'].quantile(0.25)
Q3 = flight_df['tmax'].quantile(0.75)
IQR = Q3 - Q1

# Definiowanie granic odstających wartości
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtracja danych
flight_df_outliers_cleaned = flight_df[(flight_df['tmax'] >= lower_bound) & (flight_df['tmax'] <= upper_bound)]

In [ ]:
flight_df_outliers_cleaned['tmax'].describe().round(2)

In [ ]:
flight_df_outliers_cleaned['tmax'].median()

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(flight_df_outliers_cleaned['tmax'].dropna(), bins=50, kde=True, color='grey')
plt.title('Rozkład temperatury maksymalnej (tmax)')
plt.xlabel('Temperatura maksymalna (°F)')
plt.ylabel('Częstotliwość')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x=flight_df_outliers_cleaned['tmax'], color='grey')
plt.title('Wykres pudełkowy temperatury maksymalnej (tmax)')
plt.xlabel('Temperatura maksymalna (°F)')
plt.show()

In [ ]:
flight_df_outliers_cleaned = flight_df.groupby('tmax')['is_delayed'].mean().reset_index()

In [ ]:
flight_df_outliers_cleaned['delayed_percentage'] = flight_df_outliers_cleaned['is_delayed'] * 100

In [ ]:
flight_df_outliers_cleaned = flight_df_outliers_cleaned.drop(columns=['is_delayed'])

In [ ]:
flight_df_outliers_cleaned

In [ ]:
plt.figure(figsize=(14, 7))
sns.scatterplot(
    data=flight_df_outliers_cleaned,
    x='tmax',
    y='delayed_percentage',
    alpha=0.5,  # Ustawia przezroczystość punktów
    edgecolor=None,  # Opcjonalne: usunięcie obramowania punktów
    color='grey'
)

plt.title('Temperature vs Percentage of Delayed Flights')
plt.xlabel('Temperature (°F)')
plt.ylabel('Percentage of Delayed Flights (%)')
plt.grid(True)

## Miejsce na Twój komentarz

Zarówno dla analizy uwzględniającej wartości skrajne jak i analizy nieuwzględniającej wartości skrajnych:
    - Średnia i mediana są zbliżone
    - Wzrost zmiennej tmax nie wpływa na zwiększenie odsetka opóźnionych lotów. Można zauważyć ze ta zależność jest odwrotna.

# Podsumowanie
W tej części warsztatu dokonaliśmy kompleksowej analizy posiadanego zbioru danych. Eksploracja
pozwoliła nam na zapoznanie się z cechami charakterystycznymi lotów - wiemy już, które 
zmienne mogą mieć wpływ na opóźnienia lotów, a które nie. Co warto podkreślić, skupiliśmy się na wielu
aspektach tej analizy, co otwiera potencjalnie również inne możliwości dalszej pracy nad tą bazą.

W tym momencie przejdziemy do kolejnego kroku, w którym, na podstawie tej analizy, przygotujemy 
system raportowy. Zanim jednak stworzymy dashboard, potrzebujemy zaktualizować naszą bazę danych.